# MAT8406 Regression Project: Proposal

Table of Contents
0. [Imports](#Imports)  
1. [Imput Data](#Imput_Data)  
    * [1.1 Suicide Rates Overview 1985 to 2016](#suicide_dat)  
    * [1.2 World Bank: Education Data](#edu_dat)  
    * [1.3 World Development Indicators (WDI) Data](#WID_dat)  
    


<a id=Imports></a>

## 0.0 Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import bq_helper
from functools import reduce




<a id=Imput_Data></a>

## 1.0 Input Data

Sources:
1. [Suicide Rates Overview 1985 to 2016](https://www.kaggle.com/russellyates88/suicide-rates-overview-1985-to-2016)  
    * Compares socio-economic info with suicide rates by year and country  
2. [World Bank: Education Data](https://www.kaggle.com/theworldbank/world-bank-intl-education)  
    * This dataset combines key education statistics from a variety of sources to provide a look at global literacy, spending, and access.
3. [World Development Indicators (WDI) Data](https://www.kaggle.com/bigquery/worldbank-wdi)
    * World Development Indicators Data is the primary World Bank collection of development indicators, compiled from officially-recognized international sources. It presents the most current and accurate global development data available, and includes national, regional and global estimates.  
    

<a id=suicide_dat></a>

## 1.1 Suicide Rates Overview 1985 to 2016  

In [2]:
# Reading data 
#print(os.listdir("../input"))
suicide_dat=pd.read_csv('../input/master.csv') 
# firtst 5 rows
suicide_dat.head(5)

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [3]:
# renaming cols
suicide_dat.columns = ['country', 'year', 'sex', 'age', 'suicides_no', 'population',
                       'suicides/100k pop', 'country-year', 'HDI for year',
                       ' gdp_for_year','gdp_per_capita', 'generation']
# counts of unique contries and years
print("unique contries %d and years %d." % (len(suicide_dat["country"].unique()), len(suicide_dat["year"].unique())))


unique contries 101 and years 32.


<a id=edu_dat></a>

## 1.2 World Bank: Education Data

In [4]:
# Connect to BigQuery dataset
edu_dat = bq_helper.BigQueryHelper(active_project= "bigquery-public-data",
                                   dataset_name = "world_bank_intl_education") 
# list tables in world bank: education data
edu_dat.list_tables()

Using Kaggle's public dataset BigQuery integration.


['country_series_definitions',
 'country_summary',
 'international_education',
 'series_summary']

In [5]:
# first 5 rows of international_education table in world bank: education data
edu_dat.head("international_education", num_rows=5)

,country_name,country_code,indicator_name,indicator_code,value,year
0,North America,NAC,"Population, ages 0-14 (% of total)",SP.POP.0014.TO.ZS,1.871956e+01,2016
1,North America,NAC,"Population, ages 15-64, total",SP.POP.1564.TO,2.375487e+08,2016
2,Sub-Saharan Africa (excluding high income),SSA,"Population, female",SP.POP.TOTL.FE.IN,5.178587e+08,2016
3,Upper middle income,UMC,"GDP, PPP (constant 2011 international $)",NY.GDP.MKTP.PP.KD,4.074590e+13,2016
4,Armenia,ARM,SABER: (School Health and School Feeding) Poli...,SABER.HLTH.GOAL4,2.000000e+00,2016


In [6]:
# finding indicators with highest frequency
code_count_query = """
SELECT 
    indicator_code, 
    indicator_name, 
    COUNT(country_code) AS num_rows
FROM 
    `bigquery-public-data.world_bank_intl_education.international_education`
WHERE 
    year > 1985
GROUP BY 
    indicator_code, 
    indicator_name
HAVING 
    COUNT(country_code) >= 175
ORDER BY 
    COUNT(country_code) DESC
        """

code_count_results = edu_dat.query_to_pandas_safe(code_count_query)

code_count_results.head(60)


,indicator_code,indicator_name,num_rows
0,SP.POP.TOTL,"Population, total",7379
1,SP.POP.GROW,Population growth (annual %),7373
2,SP.POP.1564.TO.ZS,"Population, ages 15-64 (% of total)",6771
3,SP.POP.0014.TO.ZS,"Population, ages 0-14 (% of total)",6761
4,SP.POP.TOTL.FE.ZS,"Population, female (% of total)",6761
5,SP.POP.TOTL.MA.ZS,"Population, male (% of total)",6761
6,SP.POP.0014.MA.IN,"Population, ages 0-14, male",6746
7,SP.POP.TOTL.FE.IN,"Population, female",6746
8,SP.POP.TOTL.MA.IN,"Population, male",6746
9,SP.POP.0014.FE.IN,"Population, ages 0-14, female",6746


In [7]:
# looking through indicators
#code_count_results.loc[180:240,:]

In [8]:
# X1: percent of goverment spending on eductation for year > 1985
query1 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SE.XPD.TOTL.GB.ZS"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
edu_spend_percent = edu_dat.query_to_pandas_safe(query1)
edu_spend_percent.columns = ['country','year','edu_spend_percent']
edu_spend_percent.head(10)

,country,year,edu_spend_percent
0,Zimbabwe,2010,8.720910
1,Zimbabwe,2013,28.660339
2,Zimbabwe,2014,30.007620
3,Zimbabwe,2012,29.199209
4,Zambia,2008,5.651800
5,Zambia,2007,6.227910
6,Zambia,2004,10.584990
7,Zambia,2000,8.352620
8,Zambia,2005,7.729730
9,"Yemen, Rep.",2008,12.489240


In [9]:
# X2: Percent annual Population growth for year > 1985
query2 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SP.POP.GROW"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
pop_growth_percent = edu_dat.query_to_pandas_safe(query2)
pop_growth_percent.columns = ['country','year','pop_growth_percent']
pop_growth_percent.head(10)


,country,year,pop_growth_percent
0,Zimbabwe,2012,2.228306
1,Zimbabwe,1987,3.471469
2,Zimbabwe,1999,1.429746
3,Zimbabwe,2013,2.309366
4,Zimbabwe,1992,2.270532
5,Zimbabwe,1996,1.733214
6,Zimbabwe,1994,1.918007
7,Zimbabwe,2009,1.842497
8,Zimbabwe,2011,2.109672
9,Zimbabwe,2010,1.976756


In [10]:
# X3: Internet users (per 100 people) for year > 1985
query3 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "IT.NET.USER.P2"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
internet_user_per100 = edu_dat.query_to_pandas_safe(query3)
internet_user_per100.columns = ['country','year','internet_user_per100']
internet_user_per100.head(10)


,country,year,internet_user_per100
0,Zimbabwe,2001,0.799846
1,Zimbabwe,2009,4.000000
2,Zimbabwe,2016,23.119989
3,Zimbabwe,2004,2.100000
4,Zimbabwe,2011,8.400000
5,Zimbabwe,1994,0.001739
6,Zimbabwe,2013,15.500000
7,Zimbabwe,2014,16.364740
8,Zimbabwe,1996,0.016790
9,Zimbabwe,2010,6.400000


In [11]:
# X5: Teachers in pre-primary education, both sexes for year > 1985
query5 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SE.PRE.TCHR"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
teachers_preprimary = edu_dat.query_to_pandas_safe(query5)
teachers_preprimary.columns = ['country','year','teachers_preprimary']
teachers_preprimary.head(10)


,country,year,teachers_preprimary
0,Zimbabwe,1995,12128.0
1,Zimbabwe,2012,10173.0
2,Zimbabwe,2003,19588.0
3,Zimbabwe,2013,9992.0
4,"Yemen, Rep.",2004,1043.0
5,"Yemen, Rep.",1992,645.0
6,"Yemen, Rep.",2011,1999.0
7,"Yemen, Rep.",1991,665.0
8,"Yemen, Rep.",2005,1179.0
9,"Yemen, Rep.",1999,754.0


In [12]:
# X6: Teachers in secondary education, both sexes for year > 1985
query6 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SE.SEC.TCHR"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
teachers_secondary = edu_dat.query_to_pandas_safe(query6)
teachers_secondary.columns = ['country','year','teachers_secondary']
teachers_secondary.head(10)


,country,year,teachers_secondary
0,Zimbabwe,1996,28254.0
1,Zimbabwe,2000,34163.0
2,Zimbabwe,1999,30572.0
3,Zimbabwe,2013,42585.0
4,Zimbabwe,2001,35416.0
5,Zimbabwe,2003,33964.0
6,Zimbabwe,1993,21403.0
7,Zimbabwe,1997,29074.0
8,Zimbabwe,2002,31957.0
9,Zimbabwe,1995,27458.0


In [13]:
# X7: Teachers in Teachers in primary education, both sexes (num..) for year > 1985
query7 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SE.PRM.TCHR"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
teachers_primary = edu_dat.query_to_pandas_safe(query7)
teachers_primary.columns = ['country','year','teachers_primary']
teachers_primary.head(10)


,country,year,teachers_primary
0,Zimbabwe,1988,57762.0
1,Zimbabwe,1989,58362.0
2,Zimbabwe,1986,58257.0
3,Zimbabwe,1997,63900.0
4,Zimbabwe,1991,58436.0
5,Zimbabwe,1990,59154.0
6,Zimbabwe,1993,61506.0
7,Zimbabwe,2001,66529.0
8,Zimbabwe,2003,61251.0
9,Zimbabwe,1992,60834.0


In [14]:
# X8: Enrolment in secondary general, both sexes (nu.) for year > 1985
query8 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SE.SEC.ENRL.GC"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
enrolment_secondary = edu_dat.query_to_pandas_safe(query8)
enrolment_secondary.columns = ['country','year','enrolment_secondary']
enrolment_secondary.head(10)


,country,year,enrolment_secondary
0,Zimbabwe,2013,957461.0
1,Zimbabwe,1993,639559.0
2,Zimbabwe,2001,866171.0
3,Zimbabwe,1997,806126.0
4,Zimbabwe,1990,661066.0
5,Zimbabwe,1999,834880.0
6,Zimbabwe,1986,537092.0
7,Zimbabwe,1996,751349.0
8,Zimbabwe,1995,711094.0
9,Zimbabwe,2003,758229.0


In [15]:
# X19: Population total for year > 1985
query19 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SP.POP.TOTL"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
population_total_edu = edu_dat.query_to_pandas_safe(query19)
population_total_edu.columns = ['country','year','population_total_edu']
population_total_edu.head(10)


,country,year,population_total_edu
0,Zimbabwe,2001,12366165.0
1,Zimbabwe,2016,16150362.0
2,Zimbabwe,1997,11709997.0
3,Zimbabwe,1985,8658857.0
4,Zimbabwe,2011,14386649.0
5,Zimbabwe,1994,11116948.0
6,Zimbabwe,2002,12500525.0
7,Zimbabwe,1999,12064537.0
8,Zimbabwe,1996,11518262.0
9,Zimbabwe,1987,9293283.0


In [16]:
# X20: Population, female for year > 1985
query20 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SP.POP.TOTL.FE.IN"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
pop_female_total = edu_dat.query_to_pandas_safe(query20)
pop_female_total.columns = ['country','year','pop_female_total']
pop_female_total.head(10)


,country,year,pop_female_total
0,Zimbabwe,2007,6833822.0
1,Zimbabwe,1995,5732167.0
2,Zimbabwe,2006,6725025.0
3,Zimbabwe,1988,4833316.0
4,Zimbabwe,2015,8099354.0
5,Zimbabwe,1987,4676227.0
6,Zimbabwe,2016,8288086.0
7,Zimbabwe,1998,6036506.0
8,Zimbabwe,2013,7729043.0
9,Zimbabwe,2001,6296890.0


In [17]:
# X21: Population, male for year > 1985
query21 = """
SELECT
  country_name,
  year,
  value
FROM
  `bigquery-public-data.world_bank_intl_education.international_education`
WHERE
  indicator_code = "SP.POP.TOTL.MA.IN"
  AND year >= 1985
GROUP BY
  country_name, year, value
ORDER BY
  country_name DESC
;
        """
pop_male_total = edu_dat.query_to_pandas_safe(query21)
pop_male_total.columns = ['country','year','pop_male_total']
pop_male_total.head(10)


,country,year,pop_male_total
0,Zimbabwe,1986,4459441.0
1,Zimbabwe,1990,5053906.0
2,Zimbabwe,2007,6496087.0
3,Zimbabwe,1985,4301352.0
4,Zimbabwe,2006,6399242.0
5,Zimbabwe,1996,5680782.0
6,Zimbabwe,1988,4770986.0
7,Zimbabwe,2009,6726570.0
8,Zimbabwe,2013,7325463.0
9,Zimbabwe,2000,6006798.0


<a id=WID_dat></a>

## 1.3 World Development Indicators (WDI) Data

In [18]:
# Connect to BigQuery dataset
WID_dat = bq_helper.BigQueryHelper(active_project="patents-public-data",
                                   dataset_name="worldbank_wdi")
# list tables in world development indicators data
WID_dat.list_tables()

Using Kaggle's public dataset BigQuery integration.


['wdi_2016']

In [19]:
# first 5 rows of wdi_2016 table in world development indicators data
WID_dat.head("wdi_2016", num_rows=5)

,year,country_name,country_code,indicator_name,indicator_code,indicator_value
0,1960,Aruba,ABW,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,0.0
1,1960,Aruba,ABW,Adjusted net national income per capita (const...,NY.ADJ.NNTY.PC.KD,0.0
2,1960,Aruba,ABW,"Adjusted net savings, excluding particulate em...",NY.ADJ.SVNX.GN.ZS,0.0
3,1960,Aruba,ABW,"Agriculture, value added (% of GDP)",NV.AGR.TOTL.ZS,0.0
4,1960,Aruba,ABW,"Air transport, registered carrier departures w...",IS.AIR.DPRT,0.0


In [20]:
###########################################
# Query cancelled; exceeds limit of 1 GB ##
###########################################
# finding indicators with highest frequency distinct
# code_count = """
# SELECT 
#     DISTINCT(indicator_code), 
#     indicator_name
# FROM 
#     `patents-public-data.worldbank_wdi.wdi_2016`
# WHERE 
#     year = 2015
# GROUP BY 
#     indicator_code, 
#     indicator_name
# ORDER BY 
#     indicator_name DESC
#         """

# code_count_WID = WID_dat.query_to_pandas_safe(code_count)

# code_count_WID.head(10)


In [21]:
# X10: Number of surgical procedures (per 100,000 population) for year > 1985
query10 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "SH.SGR.PROC.P5"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
surgical_procedures = WID_dat.query_to_pandas_safe(query10)
surgical_procedures.columns = ['country','year','surgical_procedures']
surgical_procedures.head(10)


,country,year,surgical_procedures
0,Zimbabwe,1999,0.0
1,Zimbabwe,1988,0.0
2,Zimbabwe,1997,0.0
3,Zimbabwe,2005,0.0
4,Zimbabwe,1987,0.0
5,Zimbabwe,1989,0.0
6,Zimbabwe,2016,0.0
7,Zimbabwe,2015,0.0
8,Zimbabwe,2014,0.0
9,Zimbabwe,1994,0.0


In [22]:
# X11: Population density for year > 1985
query11 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "EN.POP.DNST"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
pop_density = WID_dat.query_to_pandas_safe(query11)
pop_density.columns = ['country','year','pop_density']
pop_density.head(10)


,country,year,pop_density
0,Zimbabwe,1998,31.605899
1,Zimbabwe,1988,25.505431
2,Zimbabwe,1986,23.778917
3,Zimbabwe,1991,27.822246
4,Zimbabwe,2013,38.511289
5,Zimbabwe,2015,40.332819
6,Zimbabwe,1999,31.986607
7,Zimbabwe,2011,36.850438
8,Zimbabwe,2000,32.312217
9,Zimbabwe,1987,24.649495


In [23]:
# X12: International tourism for year > 1985
query12 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "ST.INT.TVLR.CD"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
tourism = WID_dat.query_to_pandas_safe(query12)
tourism.columns = ['country','year','tourism']
tourism.head(10)


,country,year,tourism
0,Zimbabwe,2010,0.0
1,Zimbabwe,2000,0.0
2,Zimbabwe,1992,0.0
3,Zimbabwe,1993,0.0
4,Zimbabwe,2001,0.0
5,Zimbabwe,1994,0.0
6,Zimbabwe,2008,0.0
7,Zimbabwe,1991,0.0
8,Zimbabwe,2009,0.0
9,Zimbabwe,2011,0.0


In [24]:
# X13: Mobile cellular subscriptions (per 100 people) for year > 1985
query13 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "IT.CEL.SETS.P2"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
cellular_per100 = WID_dat.query_to_pandas_safe(query13)
cellular_per100.columns = ['country','year','cellular_per100']
cellular_per100.head(10)


,country,year,cellular_per100
0,Zimbabwe,2005,5.091109
1,Zimbabwe,1997,0.047602
2,Zimbabwe,1986,0.000000
3,Zimbabwe,2007,9.620397
4,Zimbabwe,1990,0.000000
5,Zimbabwe,2003,2.869471
6,Zimbabwe,2002,2.680018
7,Zimbabwe,1994,0.000000
8,Zimbabwe,2008,12.943646
9,Zimbabwe,1992,0.000000


In [25]:
# X14: Air transport, passengers carried for year > 1985
query14 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "IS.AIR.PSGR"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
air_transport = WID_dat.query_to_pandas_safe(query14)
air_transport.columns = ['country','year','air_transport']
air_transport.head(10)


,country,year,air_transport
0,Zimbabwe,1987,459900.0
1,Zimbabwe,1998,788900.0
2,Zimbabwe,2008,264404.0
3,Zimbabwe,1996,653700.0
4,Zimbabwe,1986,451500.0
5,Zimbabwe,2014,301260.0
6,Zimbabwe,2007,254783.0
7,Zimbabwe,1988,525100.0
8,Zimbabwe,1997,790100.0
9,Zimbabwe,1994,675300.0


In [26]:
# X15: Physicians (per 1,000 people) for year > 1985
query15 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "SH.MED.PHYS.ZS"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
physicians_per1000 = WID_dat.query_to_pandas_safe(query15)
physicians_per1000.columns = ['country','year','physicians_per1000']
physicians_per1000.head(10)


,country,year,physicians_per1000
0,Zimbabwe,1991,0.0000
1,Zimbabwe,2007,0.0510
2,Zimbabwe,1987,0.1348
3,Zimbabwe,1994,0.0000
4,Zimbabwe,2000,0.0000
5,Zimbabwe,1992,0.0000
6,Zimbabwe,1993,0.0000
7,Zimbabwe,1985,0.1625
8,Zimbabwe,2008,0.0560
9,Zimbabwe,2001,0.0000


In [27]:
# X16: Nurses and midwives (per 1,000 people) for year > 1985
query16 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "SH.MED.NUMW.P3"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
nurses_per1000 = WID_dat.query_to_pandas_safe(query16)
nurses_per1000.columns = ['country','year','nurses_per1000']
nurses_per1000.head(10)


,country,year,nurses_per1000
0,Zimbabwe,2009,1.251
1,Zimbabwe,1990,1.449
2,Zimbabwe,1999,0.000
3,Zimbabwe,1988,0.000
4,Zimbabwe,2010,1.271
5,Zimbabwe,1997,0.000
6,Zimbabwe,2005,0.000
7,Zimbabwe,1987,0.000
8,Zimbabwe,1989,0.000
9,Zimbabwe,2008,1.035


In [28]:
# X17: Average precipitation in depth (mm per year) for year > 1985
query17 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "AG.LND.PRCP.MM"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
precipitation = WID_dat.query_to_pandas_safe(query17)
precipitation.columns = ['country','year','precipitation']
precipitation.head(10)


,country,year,precipitation
0,Zimbabwe,2006,0.0
1,Zimbabwe,1986,0.0
2,Zimbabwe,1990,0.0
3,Zimbabwe,2013,0.0
4,Zimbabwe,1987,657.0
5,Zimbabwe,2003,0.0
6,Zimbabwe,1999,0.0
7,Zimbabwe,1988,0.0
8,Zimbabwe,1997,657.0
9,Zimbabwe,2005,0.0


In [29]:
# X18: Population total for year > 1985
query18 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "SP.POP.TOTL"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
population_totl_WID = WID_dat.query_to_pandas_safe(query18)
population_totl_WID.columns = ['country','year','population_totl_WID']
population_totl_WID.head(10)


,country,year,population_totl_WID
0,Zimbabwe,1998,12226742.0
1,Zimbabwe,2006,13127942.0
2,Zimbabwe,2013,14898092.0
3,Zimbabwe,1993,11256512.0
4,Zimbabwe,2012,14565482.0
5,Zimbabwe,1995,11683136.0
6,Zimbabwe,1997,12059858.0
7,Zimbabwe,2002,12691431.0
8,Zimbabwe,1996,11877664.0
9,Zimbabwe,2003,12774162.0


In [30]:
# X4: Unemployment, total (% of total labor force) (modeled ILO estimate) for year > 1985
query4 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "SL.UEM.TOTL.ZS"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
Unemployment_percent_laborforce = WID_dat.query_to_pandas_safe(query4)
Unemployment_percent_laborforce.columns = ['country','year','Unemployment_percent_laborforce']
Unemployment_percent_laborforce.head(10)


,country,year,Unemployment_percent_laborforce
0,Zimbabwe,2009,6.4
1,Zimbabwe,1991,5.7
2,Zimbabwe,1998,6.5
3,Zimbabwe,2003,4.5
4,Zimbabwe,2002,4.8
5,Zimbabwe,1988,0.0
6,Zimbabwe,1987,0.0
7,Zimbabwe,2007,5.1
8,Zimbabwe,1989,0.0
9,Zimbabwe,2011,5.4


In [31]:
# X9: Unemployment, male (% of male labor force) (modeled ILO estimate) for year > 1985
query9 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "SL.UEM.TOTL.MA.ZS"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
Unemployment_percent_male = WID_dat.query_to_pandas_safe(query9)
Unemployment_percent_male.columns = ['country','year','Unemployment_percent_male']
Unemployment_percent_male.head(10)


,country,year,Unemployment_percent_male
0,Zimbabwe,2008,6.9
1,Zimbabwe,2004,4.1
2,Zimbabwe,2010,5.7
3,Zimbabwe,1997,8.6
4,Zimbabwe,2006,5.8
5,Zimbabwe,2005,4.9
6,Zimbabwe,2012,4.7
7,Zimbabwe,2003,4.3
8,Zimbabwe,2009,7.9
9,Zimbabwe,2002,4.4


In [32]:
# X22: Unemployment, female (% of male labor force) (modeled ILO estimate) for year > 1985
query22 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "SL.UEM.TOTL.FE.ZS"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
Unemployment_percent_female = WID_dat.query_to_pandas_safe(query22)
Unemployment_percent_female.columns = ['country','year','Unemployment_percent_female']
Unemployment_percent_female.head(10)


,country,year,Unemployment_percent_female
0,Zimbabwe,1992,4.4
1,Zimbabwe,1991,4.4
2,Zimbabwe,2001,5.8
3,Zimbabwe,2010,5.2
4,Zimbabwe,2007,4.3
5,Zimbabwe,1985,0.0
6,Zimbabwe,2006,4.3
7,Zimbabwe,2003,4.8
8,Zimbabwe,1998,4.8
9,Zimbabwe,2012,6.0


In [33]:
# X23: GDP for year > 1985
query23 = """
SELECT
  country_name,
  year,
  indicator_value
FROM
  `patents-public-data.worldbank_wdi.wdi_2016`
WHERE
  indicator_code = "NY.GDP.MKTP.CD"
  AND year >= 1985
GROUP BY
  country_name, year, indicator_value
ORDER BY
  country_name DESC
;
        """
GDP = WID_dat.query_to_pandas_safe(query23)
GDP.columns = ['country','year','GDP']
GDP.head(10)


,country,year,GDP
0,Zimbabwe,2006,5.443896e+09
1,Zimbabwe,2001,6.777385e+09
2,Zimbabwe,2011,1.095623e+10
3,Zimbabwe,1990,8.783817e+09
4,Zimbabwe,2007,5.291950e+09
5,Zimbabwe,2010,9.422161e+09
6,Zimbabwe,1985,5.637259e+09
7,Zimbabwe,1995,7.111271e+09
8,Zimbabwe,2014,1.419691e+10
9,Zimbabwe,1993,6.563813e+09


### Mergeing 

In [34]:
#list of dataframes from WID_dat and edu_dat queries
edu_queries = [edu_spend_percent, pop_growth_percent, internet_user_per100, teachers_preprimary, teachers_secondary,
               teachers_primary, enrolment_secondary, population_total_edu, pop_female_total, pop_male_total]
WID_queries = [pop_density, tourism, cellular_per100, air_transport, precipitation, population_totl_WID,
               Unemployment_percent_laborforce, Unemployment_percent_male, Unemployment_percent_female, GDP]

edu_merge = reduce(lambda x, y: pd.merge(x, y, on = ['country','year'],how='outer'), edu_queries)
WID_merge = reduce(lambda x, y: pd.merge(x, y, on = ['country','year'],how='outer'), WID_queries)

In [35]:
# replacing contry names with common names for all
contry_dic = {"United States" : "USA", "United States of America":"USA",
              "Bahamas, The" : "Bahamas",
              "Bolivia (Plurinational State of)" : "Bolivia",
              "Virgin Islands (U.S.)" : "Virgin Islands (USA)",
              "Brunei Darussalam" : "Brunei",
              "Cabo Verde" : "Cape Verde",
              "Congo" : "Republic of Congo", "Congo, Rep." : "Republic of Congo",
              "Congo, Dem. Rep." : "Democratic Republic of the Congo",
              "Côte d'Ivoire" : "Cote d'Ivoire",
              "Czechia" : "Czech Republic",
              "Democratic People's Republic of Korea" : "North Korea", "Korea, Dem. People’s Rep." : "North Korea",
              "Korea, Dem. People<U+FFFD>s Rep." : "North Korea",
              "Republic of Korea" : "South Korea", "Korea, Rep." : "South Korea",
              "Kyrgyz Republic" : "Kyrgyzstan",
              "Egypt, Arab Rep." : "Egypt",
              "Eswatini" : "Swaziland",
              "Gambia, The" : "Gambia",
              "Hong Kong SAR, China" : "Hong Kong SAR",
              "Iran, Islamic Rep." : "Iran", "Iran (Islamic Rep of)" : "Iran", "Iran (Islamic Republic of)" : "Iran",
              "Lao PDR" : "Laos", "Lao People's Democratic Republic" : "Laos",
              "Macedonia, FYR" : "Macedonia", "TFYR Macedonia" : "Macedonia","Republic of North Macedonia":"Macedonia",
              "Micronesia, Fed. Sts." : "Micronesia", "Micronesia (Federated States of)" : "Micronesia",
              "Republic of Moldova" : "Moldova", "Russian Federation" : "Russia",
              "St. Kitts and Nevis" : "Saint Kitts and Nevis",
              "Slovak Republic" : "Slovakia",
              "Saint Vincent and the Grenadines" : "Saint Vincent and Grenadines",
              "St. Vincent and the Grenadines" : "Saint Vincent and Grenadines",
              "St. Lucia" : "Saint Lucia",
              "Syrian Arab Republic" : "Syria",
              "United Kingdom of Great Britain and Northern Ireland" : "UK", "United Kingdom" : "UK",
              "United Republic of Tanzania" : "Tanzania",
              "Venezuela (Bolivarian Republic of)" : "Venezuela", "Venezuela, RB" : "Venezuela",
              "Viet Nam" : "Vietnam",
              "Yemen, Rep." : "Yemen"
             }
# map(lambda x : x.replace(to_replace=contry_dic, value=None, inplace=True), queries)
edu_merge.replace(to_replace=contry_dic, value=None, inplace=True)
WID_merge.replace(to_replace=contry_dic, value=None, inplace=True)
suicide_dat.replace(to_replace=contry_dic, value=None, inplace=True)

In [36]:
# outer merge of all df.query
WID_edu_merge = pd.merge(edu_merge, WID_merge, on = ['country','year'],how='outer')


In [37]:
merge_suicide_WID_edu = pd.merge(suicide_dat, WID_edu_merge, on = ['country','year'])
print("WID edu suicide merge shape %s unique contries %d and years %d." % (str(merge_suicide_WID_edu.shape), len(merge_suicide_WID_edu["country"].unique()), len(merge_suicide_WID_edu["year"].unique())))

WID edu suicide merge shape (27808, 32) unique contries 100 and years 32.


In [38]:
merge_suicide_WID_edu.to_csv('MAT8406Regression.csv', index = False)
WID_edu_merge.to_csv('WID_edu_merge.csv', index = False)